In [64]:
import ast
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras, feature_column
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout

In [77]:
df = pd.read_csv('data/task_environments_2021-07-15_transformed.csv', header=0, index_col=0)

In [78]:
df.columns

Index(['sequence', 'seq1', 'seq2', 'seq3', 'seq4', 'seq5', 'seq6', 'seq7',
       'seq8', 'seq9',
       ...
       'coordinates_l.z', 'coordinates_a.x', 'coordinates_a.y',
       'coordinates_a.z', 'coordinates_h.x', 'coordinates_h.y',
       'coordinates_h.z', 'coordinates_q.x', 'coordinates_q.y',
       'coordinates_q.z'],
      dtype='object', length=216)

In [79]:
train, test = train_test_split(df, test_size=0.2)

In [80]:
print(len(train), len(test))

152 38


In [81]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('sequence')
    dataset = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
        
    dataset = dataset.batch(batch_size)
    
    return dataset

In [82]:
batch_size = 10

In [83]:
train_ds = df_to_dataset(train, batch_size=batch_size)

ValueError: Can't convert Python sequence with mixed types to Tensor.

In [84]:
def create_input_data(dataframe):
    feature_columns = []
    #target = []
    
    #target = dataframe.pop('sequence')
    
    for header in dataframe.columns:
        if 'coord' in header:
            feature_columns.append(feature_column.numeric_column(header))
        elif 'containment' in header or 'food' in header or 'mid' in header or \
        'strong' in header:
            header_col = feature_column.categorical_column_with_vocabulary_list(
            header, ['True', 'False'])
            header_embedding = feature_column.embedding_column(header_col, dimension=10)
            feature_columns.append(header_embedding)
                
    return feature_columns#, target

In [86]:
feature_cols = create_input_data(df)

In [87]:
feature_layer = tf.keras.layers.DenseFeatures(feature_cols)